In [ ]:
import pandas as pd
from catboost import CatBoostRegressor
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import folium
import math
from datetime import timedelta
import random

#Thư viện quy đổi tọa độ -> met
from haversine import haversine, Unit
from geographiclib.geodesic import Geodesic
geod = Geodesic.WGS84

import osmnx as ox, geopandas as gpd
import seaborn as sns
%matplotlib inline
ox.config(log_console=True, use_cache=True)

In [ ]:
data = pd.read_csv('data_pre.csv')

In [ ]:
def add_markers(gdf):
    f = folium.Figure(height = 800)
    mapobj = folium.Map([np.median(gdf.x), np.median(gdf.y)], zoom_start = 15, tiles='Cartodb dark_matter')
    mapobj.add_to(f)
    coords = []
    for i, row in gdf.iterrows():
        coords.append([row.x, row.y])
    for coord in coords:
        folium.CircleMarker(location = coord,
                            radius = 2.5, 
                            fill = True,
                            fill_color = '#F50057',
                            fill_opacity = 0.75,
                            color = 'whitesmoke',
                            weight = 0.5).add_to(mapobj)
    return mapobj

In [ ]:
add_markers(data[data.vehicle == ''])

In [ ]:
colors = [
    'red',
    'yellow',
    'blue',
    'lightred',
    'orange',
    'green',
    'lightgreen',
    'purple',
    'pink']

def add_point(mapobj, df, colors):
    #Nạp x,y từ dataframe vào list coords
    coords = list(zip(df.x, df.y))
    
    #Hiển thị trên mapobj
    for coord in coords:
        folium.CircleMarker(location = coord,
                            radius = 1.5, 
                            fill = True,
                            fill_opacity = 0.75,
                            color = colors,
                            weight = 0.01).add_to(mapobj)
        
        
def add_lines(mapobj, df, colors):
    coords = list(zip(df.x, df.y))
    folium.PolyLine(coords, color=colors, weight=1, opacity=1).add_to(mapobj)   
       
    
'''
Hàm hiển thị map.
Tùy chọn:
- df: dataframe input
- n: Số lộ trình cần hiển thị
- type_map: dạng đường (1) và dạng điểm (0)
'''
def show_n_route(df, n, type_map):
    #Khởi tạo bản đồ mapobj
    f = folium.Figure(height = 800)
    mapobj = folium.Map([np.median(df.x), np.median(df.y)], zoom_start = 15, tiles='Cartodb dark_matter')
    mapobj.add_to(f)


    #Hiển thị n lộ trình đầu tiên
    index_route = df[df['level_1'] == 0].index
    
    if n == -1:
        n = len(index_route)-1
    
    #Biến đếm màu
    t = 0
    if len(index_route)==1:
        if type_map == 0:
            add_point(mapobj, df, colors[t])
        else:
            add_lines(mapobj, df, colors[t])
        t += 1
    for i in range(n):
        df_temp = df.loc[index_route[i]:index_route[i + 1] - 1,['x', 'y']]
        if type_map == 0:
            add_point(mapobj, df_temp, colors[t])
        else:
            add_lines(mapobj, df_temp, colors[t])
        t += 1
        if t == 9:
            t = 0
        
    return mapobj

In [ ]:
# lấy dữ liệu của n xe
df = pd.DataFrame()
for i in list(set(data.vehicle))[6:7]:
    df = df.append(data[data.vehicle == i], ignore_index = True)
df
#show_n_route(data.loc[:100], 0, 1)

In [ ]:
show_n_route(df,1,0)

In [ ]:
def show_n_route_use_ox(df, n, type_map):
    #Khởi tạo bản đồ mapobj
    f = folium.Figure(height = 800)
    mapobj = folium.Map([np.median(df.x), np.median(df.y)], zoom_start = 15, tiles='Cartodb dark_matter')
    mapobj.add_to(f)


    #Hiển thị n lộ trình đầu tiên
    index_route = df[df['level_1'] == 0].index
    
    #Biến đếm màu
    t = 0
    if len(index_route)==1:
        if type_map == 0:
            add_point(mapobj, df, colors[t])
        else:
            add_lines(mapobj, df, colors[t])
        t += 1
    for i in range(len(index_route)-1):
        df_temp = df.loc[index_route[i]:index_route[i + 1] - 1,['x', 'y']]
        if type_map == 0:
            add_point(mapobj, df_temp, colors[t])
        else:
            add_lines(mapobj, df_temp, colors[t])
        t += 1
        if t == 9:
            t = 0
        
    return mapobj

In [ ]:
class GEPoint2D:

        def __init__(self, px, py):
            self.x = px
            self.y = py
        

def Distance(p1, p2):

    # GEPoint2D p1, GEPoint2D p2

    if p1 == p2:
        return 0

    dbLat1InRad = p1.y * (math.pi / 180.0)
    dbLong1InRad = p1.x * (math.pi / 180.0)
    dbLat2InRad = p2.y * (math.pi / 180.0)
    dbLong2InRad = p2.x * (math.pi / 180.0)

    dbTheta = p1.x - p2.x
    dbThetaInRad = dbTheta * (math.pi / 180.0)

    a = math.sin(dbLat1InRad)
    b = math.sin(dbLat2InRad)
    c = math.cos(dbLat1InRad)
    d = math.cos(dbLat2InRad)
    e = math.cos(dbThetaInRad)
    f = a * b + c * d * e

    if f > 1:
        f = 1
    if f < -1:
        f = -1

    dbDistInRad = math.acos(f)
    dbDist = dbDistInRad * 180.0 / math.pi
    dbDist = dbDist * 60.0 * 1.1515
    dbDist = dbDist * 1.609344 * 1000

    return dbDist

def direct_point(A,A_next): 
    return geod.Inverse(float(A.x),float(A.y),float(A_next.x),float(A_next.y))['azi1']

def Line2D(p1,p2):
    #a = p1.y - p2.y
    #b = p2.x - p1.x
    #c = -a * p1.x - b * p1.y
    x = (p1.x ,p1.y)
    y = (p2.x, p2.y)
    coef = np.polyfit(x, y, 1)
    a = coef[0]
    b = coef[1]
    c = a*x[0] + b*x[1]
    
    return a, b, c

class Point2D:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
class Vector2D:
    
    def __init__(self,point1, point2):
        self.p1 = point1
        self.p2 = point2
        self.vx = point1.x - point2.x
        self.vy = point1.y - point2.y
        
    def Distance(self,p):
        a, b, c = Line2D(point2,p)
        distance = abs(a*p.x + b*p.y + c)/((a*a+b*b)**(1/2))
        distance = distance / 0.00000833
        return distance
    
    def Angle(self,v1):
        self.cost = self.vx * self.vx + self.vx * self.vy
        
        
def shortest_distance(x1, y1, a, b, c):
    d = abs((a * x1 + b * y1 + c)) / (math.sqrt(a * a + b * b))
    return d

def distance_p3_with_line(p1, p2, p3):
    A =  GEPoint2D(p1.x,p1.y)
    B = GEPoint2D(p2.x,p2.y)
    C = GEPoint2D(p3.x,p3.y)
    a = Distance(B,C)
    b = Distance(A,C)
    c = Distance(A,B)
    cosAngleA = (b*b +c*c -a*a) / (2*b*c)
    angleA = math.acos(cosAngleA)
    distance_p3 = b * math.sin(angleA)
    return distance_p3


p1 = Point2D(106.696495,10.766200)

In [ ]:
df1 = data[data.vehicle == '15C03351']
df1

#print(ox.distance.euclidean_dist_vec(df1.iloc[0].x, df1.iloc[0].y, df1.iloc[1].x, df1.iloc[1].y) / 0.00000833)
#print(haversine((df1.iloc[0].x, df1.iloc[0].y), (df1.iloc[1].x, df1.iloc[1].y)))

In [ ]:
df1

In [ ]:


def route(df):
    '''
    
    '''
    # Danh sách các số xe
    list_vehicle = list(set(df.vehicle))
    list_all_route_vehicle = []
    for vehicle in list_vehicle:
        # DataFrame các lộ trình của từng xe
        df1 = df[df.vehicle == vehicle]
        # danh sách các điểm đầu của từng lộ trình của mỗi xe
        list_idx_route = df1[df1.level_1 == 0].index
        # Khởi tạo list danh sách các lộ trình
        list_route  = []
        vector = 0
        # Sử dụng vòng for với danh sách các điểm đầu lộ trình
        for i in range(len(list_idx_route)):
            # nếu lộ trình ở cuối danh sách thì list_route append phần còn lại trong dataFrame lộ trình df1
            if i == len(list_idx_route) - 1:
                list_route.append(df1.loc[list_idx_route[i]:])
            else:
            # nếu lộ trình không ở cuối thì list_route append điểm đầu lộ trình và điểm đầu lộ trình tiếp theo - 1
                list_route.append(df1.loc[list_idx_route[i]:list_idx_route[i+1]-1])
                
        # Nối các lộ trình của từng xe
        for i in range(len(list_route)-1,-1,-1):
            # nếu lộ trình đang xét chỉ có 1 điểm thì ta sẽ nối nó vào lộ trình trước đó
            if len(list_route[i]) == 1:
                if len(list_route[i-1]) == 1:
                    list_route[i]['level_1'] = 1 #max(list_route[i-1].level_1) + 1
                    list_route[i-1] = list_route[i-1].append(list_route[i])
                else:
                    d = distance_p3_with_line(list_route[i-1].iloc[-2], list_route[i-1].iloc[-1], list_route[i].iloc[0])
                    print(d)
                    if d < 7:
                        idx = list_route[i].index
                        list_route[i] = list_route[i].reset_index(drop = True)
                        list_route[i].loc[[0],'level_1'] = max(list_route[i-1].level_1) + 1
                        list_route[i] = list_route[i].set_index(idx)
                        list_route[i-1] = list_route[i-1].append(list_route[i])
            
            # nếu lộ trình trước của lộ trình đang xét chỉ có 1 điểm 
            # ta xét vector lộ trình hiện tại có đi qua điểm của lộ trình trước đó ko
            elif len(list_route[i-1]) == 1:
                d= distance_p3_with_line(list_route[i].iloc[0], list_route[i].iloc[1], list_route[i-1].iloc[0])
                print(d)
                if d < 7:
                    max_level = max(list_route[i-1].level_1) + 1
                    for j in range(len(list_route[i])):
                        idx = list_route[i].index
                        list_route[i] = list_route[i].reset_index(drop = True)
                        list_route[i].loc[[j],'level_1'] = max_level
                        list_route[i] = list_route[i].set_index(idx)
                        list_route[i-1] = list_route[i-1].append(list_route[i].iloc[j])
                        max_level += 1
                        
            # nếu cả 2 lộ trình đều chứa nhiều điểm ta xét 2 vector của chúng
            else:
                vector_a = direct_point(list_route[i].iloc[0], list_route[i].iloc[1])
                vector_b = direct_point(list_route[i-1].iloc[-2], list_route[i-1].iloc[-1])
                if abs(vector_a - vector_b) < 5:
                    max_level = max(list_route[i-1].level_1) + 1
                    for j in range(len(list_route[i])):
                        idx = list_route[i].index
                        list_route[i] = list_route[i].reset_index(drop=True)
                        
                        list_route[i].loc[[j],'level_1'] = max_level
                        list_route[i] = list_route[i].set_index(idx)
                        list_route[i-1] = list_route[i-1].append(list_route[i].iloc[j])
                        max_level += 1
                
                
    #  list_all_route_vehicle.append(list_route)
    return list_route[0]
list_route1 = route(df1)

1.0913957891587616
0.45807108914105316
0.22182954813046502
18.97179256762462


In [ ]:
list_route1

In [ ]:
df1

In [ ]:
show_n_route(list_route1,0,1)

In [ ]:
direct_point(df.iloc[2],df.iloc[3])

-146.0398584131737

In [ ]:
list_idx_route = df1[df1.level_1 == 0].index
# Khởi tạo list danh sách các lộ trình
list_route  = []

vector = 0
# Sử dụng vòng for với danh sách các điểm đầu lộ trình
for i in range(len(list_idx_route)):
    # nếu lộ trình ở cuối danh sách thì list_route append phần còn lại trong dataFrame lộ trình df1
    if i == len(list_idx_route) - 1:
        list_route.append(df1.loc[list_idx_route[i]:])
    else:
    # nếu lộ trình không ở cuối thì list_route append điểm đầu lộ trình và điểm đầu lộ trình tiếp theo - 1
        list_route.append(df1.loc[list_idx_route[i]:list_idx_route[i+1]-1])
#print(direct_point(list_route[0].iloc[0], list_route[0].iloc[1]))
#print(direct_point(list_route[0].iloc[-2], list_route[0].iloc[-1]))
#a,b,c = Line2D(list_route[0].iloc[0], list_route[0].iloc[1])
#shortest_distance(list_route[0].iloc[2].x, list_route[0].iloc[2].y, a, b, c)
distance_p3_with_line(df.iloc[0], df.iloc[1], df.iloc[2])

89.84492047335047

In [ ]:
add_markers(df[:3])

In [ ]:
id = pd.Series(list_route1.index)
list_route1 = list_route1.reset_index(drop=True)
list_route1.loc[[1],'level_1'] = 1
list_route1 = list_route1.set_index(id)
list_route1

In [ ]:
id

0     153
1     154
2     155
3     156
4     157
5     158
6     159
7     160
8     161
9     162
10    163
11    164
12    165
13    166
14    167
15    168
16    169
dtype: int64

In [ ]:
list_route1.iloc[1]

vehicle                        15C03351
level_1                               1
datetime            2018-04-24 19:08:08
speed                         16.000000
y                            106.683750
x                             10.767620
heading                       42.363156
vehicleType                         500
time_interval                 36.000000
distance                     237.527808
heading_interval               1.639773
Name: 154, dtype: object

In [ ]:
show_n_route_use_ox(df,-1,type_map = 0)

In [ ]:
list_route